In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Activation, Dense,Flatten, Dropout, Input
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
import keras
from keras.losses import categorical_crossentropy

Using TensorFlow backend.


In [2]:
lmbda = 0.1
batch_size =128
epochs = 200
seed = 0
learning_rate = 0.1
data_augmentation = False
cutout = 16
budget = 0.3

In [3]:
VGG13 = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

input_shape = (32, 32, 3)
num_classes = 10
name = 1

input_tensor = Input(shape=input_shape, name='input')
net = input_tensor

for x in VGG13:
    if x == 'M':
        net = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool'+str(name))(net)
        name += 1
    else:
        net = ZeroPadding2D(padding=(1, 1), name='zero'+str(name))(net)
        net = Conv2D(64, (3, 3), padding='same', name='conv'+str(name))(net)
        net = BatchNormalization(name='norm'+str(name))(net)
        net = Activation('relu', name='acti'+str(name))(net)
        name += 1
        
net = AveragePooling2D(pool_size=(1, 1), strides=(1,1), padding='same', name='avepool')(net)
net = Flatten(name='flatten')(net)
net = Dense(512, name='dense')(net)
prediction = Dense(num_classes, name='prediction')(net)
confidence = Dense(1, name='confidence')(net)

model = Model(input_tensor, (prediction, confidence))

In [11]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [12]:
def preprocess(images, length, train=True):
    
    # Normalize
    mean = [125.3, 123.0, 113.9]
    std = [63.0, 62.1, 66.7]
    images = ((images - mean) / std)
    
    # Cutout
    if train:
        h = images[0].shape[0]
        w = images[0].shape[1]
        mask = np.ones((h, w), np.float32)
        y = np.random.randint(h)
        x = np.random.randint(w)
        y1 = int(np.clip(y - length / 2, 0, h))
        y2 = int(np.clip(y + length / 2, 0, h))
        x1 = int(np.clip(x - length / 2, 0, w))
        x2 = int(np.clip(x + length / 2, 0, w))
        mask[y1: y2, x1: x2] = 0.
        tensor = np.append(mask, mask)
        tensor = np.append(tensor, mask)
        tensor = tensor.reshape((32, 32, 3))
        images = images * tensor
        
    return images

In [14]:
images = preprocess(x_train, cutout, True)
x_test = preprocess(x_test, cutout, False)

In [15]:
labels = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

In [16]:
images = images[:batch_size]
labels = labels[:batch_size]

In [22]:
y_pred = model.predict(images, batch_size=batch_size, verbose=1)

128/128 [==============================] - 0s 113us/step


In [41]:
import tensorflow as tf

In [46]:
tf.Variable(y_pred[0])

<tf.Variable 'Variable_4:0' shape=(128, 10) dtype=float32_ref>

In [23]:
prediction = K.variable(y_pred[0])
confidence = K.variable(y_pred[1])

In [26]:
# clipメソッドでインプットを範囲内に収める
eps = 1e-12
pred_original = K.clip(prediction, 0. + eps, 1. - eps)
confidence = K.clip(confidence, 0. + eps, 1. - eps)

In [29]:
# 予測値の補正を行う（ヒント部分）
# Randomly set half of the confidences to 1 (i.e. no hints)
b = K.variable(np.random.randint(0,2,confidence.shape))

In [31]:
# confを設定
conf = confidence * b + (1 - b)

In [39]:
origin_tensor = K.variable(K.get_value(conf))

In [40]:
type(origin_tensor)

tensorflow.python.ops.variables.Variable